In [2]:
import numpy as np
import tensorflow as tf
import scipy
from tensorflow import keras
from tensorflow.keras import layers

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from tensorflow.keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

import matplotlib.pyplot as plt
from scipy import stats

from tqdm.notebook import tqdm

## Extracting features from VGG16

In [3]:
img_size = 224
mobilenet = MobileNet(weights='imagenet', include_top=True, pooling='max', input_shape = (img_size, img_size, 3))
mobilenet.summary()

17227776/17225924 [==============================] - 1s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________

In [4]:
op = mobilenet.get_layer('reshape_2').output
ip = mobilenet.input

basemodel = keras.Model(
    inputs=ip,
    outputs=op,
)

basemodel.trainable=False

In [5]:
images = np.empty((50, img_size, img_size, 3))
for i in range(50):
  image = load_img("data/" + str(i+1) +".jpg", target_size=(img_size, img_size))
  image = img_to_array(image)

  for j in range(img_size):
    for k in range(img_size):
      g = (image[j,k,0] + image[j,k,1] + image[j,k,2])/3
      image[j,k,0] = g
      image[j,k,1] = g
      image[j,k,2] = g

  images[i,:,:,:] = image

images = preprocess_input(images)
features = basemodel.predict(images)

colors = np.empty((5, img_size, img_size, 3))
for i in range(5):
  image = load_img("data/C" + str(i) + ".png", target_size=(img_size, img_size))
  image = img_to_array(image)
  colors[i,:,:,:] = image

colors = preprocess_input(colors)
colorfeatures = basemodel.predict(colors)

features = features.reshape(50,-1)
colorfeatures = colorfeatures.reshape(5,-1)

data = np.genfromtxt('data/bandw.csv', skip_header = 6, delimiter=',')
humanpred = np.zeros((50,5))
for i in range(50):
  for j in range(56):
    color = int(data[j, i]);
    humanpred[i, color] = humanpred[i, color]+1

prob = humanpred/56

In [6]:
inputimages = np.zeros((250,features.shape[1]))
inputcolors = np.zeros((250,features.shape[1]))
outputprob = np.zeros((250))

for i in range(50):
  for j in range(5):
    inputimages[5*i+j,:] = features[i,:]

for i in range(5):
  for j in range(50):
    inputcolors[5*j+i] = colorfeatures[i, :]

outputprob = prob.reshape(250)

## Model

In [7]:
def get_model(op_features=75):
  imageinputs = keras.Input(shape=(features.shape[1]), name="imageinputs")
  colorinputs = keras.Input(shape=(features.shape[1]), name="colorinputs")
  dense = layers.Dense(op_features, name = "linearlayer", activation='relu')
  imageresults = dense(imageinputs)
  colorresults = dense(colorinputs)
  result = tf.keras.layers.Dot(axes=1, normalize=True, name="dot")([imageresults, colorresults])

  model = keras.Model(
      inputs=[imageinputs, colorinputs],
      outputs=[result],
  )
  model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(lr=0.001), metrics = ['mean_squared_error'])
  
  return model

def get_corr(output, outputprob, vs, ve):
  x = np.copy(output.reshape(50,5))

  for i in range(50):
    rowsum = np.sum(x[i,:])
    for j in range(5):
      x[i,j] = x[i,j]/rowsum

  y = np.copy(outputprob.reshape(50,5))
  testx = x.reshape(-1)[vs:ve]
  testy = y.reshape(-1)[vs:ve]

  return scipy.stats.pearsonr(testx, testy)[0], scipy.stats.pearsonr(testx, testy)[1], testx

In [8]:
Rp = []
pvalue = []

for i in tqdm(range(50)):
  corr = [] #This contains correlation corresponding to specific validation data
  pred = np.zeros(250) #This will contain the final prediction for each of the loop. Total 5 subset of validation 50 each.

  # 5-fold cross validation 
  for vs in [0, 50, 100, 150, 200]:
    ve = vs + 50
    model = get_model(op_features=75)
    history = model.fit(
        {"imageinputs": inputimages[[*range(vs)] + [*range(ve, 250)]], "colorinputs": inputcolors[[*range(vs)] + [*range(ve, 250)]]},
        {"dot": outputprob[[*range(vs)] + [*range(ve, 250)]]},
        epochs=30,
        batch_size=10,
        shuffle=True,
        verbose=0,
    )
    output = model.predict({"imageinputs": inputimages, "colorinputs": inputcolors})
    R, _, pred[vs:ve] = get_corr(output, outputprob, vs, ve)
    corr.append(R)

  temp = get_corr(pred, outputprob, 0, 250)
  Rp.append(temp[0])
  pvalue.append(temp[1])

In [9]:
print(np.mean(Rp))
print(np.std(Rp))

0.5327758527035148
0.027333013270097555


In [10]:
Rp

[0.4643335900208154,
 0.5916653521154563,
 0.5185870310582797,
 0.5194349553736115,
 0.566549185732745,
 0.5323723131839688,
 0.5301795700501558,
 0.5292724826814267,
 0.5083769604787874,
 0.5109697858227084,
 0.5275895929812386,
 0.561507418772449,
 0.5524841454085143,
 0.5770596118034835,
 0.562160777933095,
 0.5527223198328242,
 0.5155137575772851,
 0.5424156093872838,
 0.5151967030571446,
 0.5112222698507539,
 0.5204235755909852,
 0.5201449275886896,
 0.540954860066318,
 0.4895335400393026,
 0.5228167268189827,
 0.5671034327875943,
 0.5151137165417249,
 0.5505974043268088,
 0.5774490916412615,
 0.5415809113311139,
 0.5333043858823484,
 0.5293366932808451,
 0.5271670261274783,
 0.5696856992092784,
 0.5236238527117749,
 0.5313123367647407,
 0.5005779520847097,
 0.4867698910310284,
 0.5486593933219697,
 0.5211652397182673,
 0.5518766496361599,
 0.548957560186373,
 0.5653071893482361,
 0.5428451255826859,
 0.5383467223028208,
 0.5587276412948893,
 0.49227404447644313,
 0.47167140926289

In [11]:
print(np.mean(pvalue))
print(np.std(pvalue))

2.525612731288252e-16
1.3156687381115543e-15
